In [1]:
import torch
import torchvision.datasets as datasets
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader
from torch import nn


In [2]:
# Download training data from open datasets.
training_data = datasets.CIFAR10(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.CIFAR10(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

100%|██████████| 170M/170M [00:07<00:00, 21.5MB/s]


In [3]:
#Personal note. The set contains a set of image and label. Access image is data[i][0] and access label is data[i][1]
print(training_data.classes[training_data[0][1]] )


frog


In [4]:
batch_size = 64

#Create data loaders
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 3, 32, 32])
Shape of y: torch.Size([64]) torch.int64


In [5]:
device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
print(f"Using {device} device")

#Define model
class NeuralNetwork(nn.Module):
  def __init__(self):
    super().__init__()
    self.flatten = nn.Flatten()
    self.linear_relu_stack = nn.Sequential(
      nn.Linear(32*32*3, 512),
      nn.ReLU(),
      nn.Linear(512, 512),
      nn.ReLU(),
      nn.Linear(512, 10)
    )

  def forward(self, x):
    x = self.flatten(x)
    logits = self.linear_relu_stack(x)
    return logits

model = NeuralNetwork().to(device)
print(model)

Using cuda device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=3072, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [6]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [7]:
def train(dataloader, model , loss_fn, optimizer):
  size = len(dataloader.dataset)
  model.train()
  for batch, (X, y) in enumerate(dataloader):
    X, y = X.to(device), y.to(device)

    #Compute prediction error
    pred = model(X)
    loss = loss_fn(pred, y)

    #Backpropagation
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

    if batch % 100 == 0:
      loss, current = loss.item(), batch * len(X)
      print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")


In [8]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [9]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.303422 [    0/50000]
loss: 2.300388 [ 6400/50000]
loss: 2.295327 [12800/50000]
loss: 2.290386 [19200/50000]
loss: 2.285543 [25600/50000]
loss: 2.280494 [32000/50000]
loss: 2.278836 [38400/50000]
loss: 2.284734 [44800/50000]
Test Error: 
 Accuracy: 15.7%, Avg loss: 2.269211 

Epoch 2
-------------------------------
loss: 2.283875 [    0/50000]
loss: 2.280887 [ 6400/50000]
loss: 2.250039 [12800/50000]
loss: 2.267228 [19200/50000]
loss: 2.244891 [25600/50000]
loss: 2.244555 [32000/50000]
loss: 2.258569 [38400/50000]
loss: 2.248553 [44800/50000]
Test Error: 
 Accuracy: 20.8%, Avg loss: 2.233148 

Epoch 3
-------------------------------
loss: 2.257879 [    0/50000]
loss: 2.248911 [ 6400/50000]
loss: 2.196619 [12800/50000]
loss: 2.238264 [19200/50000]
loss: 2.195129 [25600/50000]
loss: 2.197503 [32000/50000]
loss: 2.230970 [38400/50000]
loss: 2.194691 [44800/50000]
Test Error: 
 Accuracy: 23.3%, Avg loss: 2.183704 

Epoch 4
--------------------